In [ ]:
import os
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
import access

###############################################################################################
#            CONVERTE O ARQUIVO PARA .CSV E SALVA EM OUTRA PASTA O ARQUIVO ORIGINAL           #
###############################################################################################
def xlsx_to_csv(input_file, output_file, processed_folder):
    # Carregue o arquivo Excel usando pandas
    df = pd.read_excel(input_file)

    # Salve o DataFrame como um arquivo CSV separado por ponto e vírgula
    df.to_csv(output_file, index=False, sep=';')

    print(f'O arquivo {input_file} foi convertido para {output_file}.')

    # Mova o arquivo original para a pasta de processados com '_processado' no nome
    if not os.path.exists(processed_folder):
        os.makedirs(processed_folder)

    # Construa o caminho completo para o arquivo original na pasta de processados
    processed_original_file_path = os.path.join(processed_folder, os.path.basename(input_file.replace('.xlsx', '_processado.xlsx')))

    # Mova o arquivo original para a pasta de processados
    os.rename(input_file, processed_original_file_path)

    print(f'O arquivo original foi movido para a pasta {processed_folder}.')

def convert_files_in_folder(folder_path, processed_folder):
    # Liste todos os arquivos na pasta de origem
    files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]

    for file in files:
        # Construa o caminho completo para o arquivo de entrada
        input_file_path = os.path.join(folder_path, file)

        # Construa o caminho completo para o arquivo de saída
        output_file_path = os.path.join(folder_path, file.replace('.xlsx', '.csv'))

        # Chame a função para realizar a conversão e mover o arquivo
        xlsx_to_csv(input_file_path, output_file_path, processed_folder)

###############################################################################################
#                               CARGA DO ARQUIVO NA TRANSIENT                                 #
###############################################################################################

# Substitua com o caminho da sua pasta de origem e a pasta de processados
source_folder = access.source_folder
processed_folder = access.processed_folder

# Chame a função para converter todos os arquivos na pasta de origem
convert_files_in_folder(source_folder, processed_folder)

# Caminho para a chave do IAM
key_path = access.key_path

# Carregar credenciais
credentials = service_account.Credentials.from_service_account_file(key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"])

def load_to_bigquery(data_frame, dataset_id, table_id, credentials):
    # Inicializar cliente BigQuery
    client = bigquery.Client(credentials=credentials)

    # Criar referência para a tabela de destino
    table_ref = client.dataset(dataset_id).table(table_id)

    # Configurações do job de carregamento
    job_config = bigquery.LoadJobConfig(
        skip_leading_rows=0,
        source_format=bigquery.SourceFormat.CSV,
    )

    # Carregar dados na tabela
    job = client.load_table_from_dataframe(data_frame, table_ref, job_config=job_config)
    job.result()  # Aguardar término do job

    print(f'Dados carregados na tabela {table_id} no BigQuery.')

    # Imprimir informações sobre a carga
  
    print(f'Número de linhas carregadas: {len(data_frame)}')

def delete_file(file_path):
    try:
        os.remove(file_path)
        print(f'Arquivo {file_path} excluído.')
    except Exception as e:
        print(f"Erro ao excluir o arquivo {file_path}: {str(e)}")


###############################################################################################
#                               CARGA DO ARQUIVO NA TRANSIENT                                 #
###############################################################################################

print('truncate transient - nova carga ')
# Construir a instrução SQL CREATE OR REPLACE --> TRUNCATE STG  -- VERSÃO GRATIS NÃO DEIXA FAZER TRUNCATE 
sql_create_replace_transient = f"""

CREATE OR REPLACE TABLE transient.stg_base_vendas (
    ID_MARCA INT64,
    MARCA STRING,
    ID_LINHA INT64,
    LINHA STRING,
    DATA_VENDA DATE,
    QTD_VENDA INT64
);

"""

# Executar a instrução SQL CREATE OR REPLACE
client = bigquery.Client(credentials=credentials)
query_job_create_replace_transient = client.query(sql_create_replace_transient)
query_job_create_replace_transient.result()

# Substitua com as informações do seu projeto, dataset e tabela no BigQuery
dataset_id_transient = 'transient'
table_id_transient = 'stg_base_vendas'

# Iterar sobre os arquivos no diretório de origem
files_transient = [f for f in os.listdir(source_folder) if f.endswith('.csv')]

for file in files_transient:
    # Caminho completo para o arquivo de entrada
    input_file_path = os.path.join(source_folder, file)

    # Carregar dados para o BigQuery
    df_transient = pd.read_csv(input_file_path, delimiter=';')
    load_to_bigquery(df_transient, dataset_id_transient, table_id_transient, credentials)

    # Excluir o arquivo do diretório
    delete_file(input_file_path)

###############################################################################################
#                               CARGA DO ARQUIVO NA RAW                                       #
###############################################################################################

print('truncate raw - nova carga ')
# Construir a instrução SQL CREATE OR REPLACE --> TRUNCATE STG  -- VERSÃO GRATIS NÃO DEIXA FAZER TRUNCATE 
sql_create_replace_raw = f"""

CREATE OR REPLACE TABLE raw_data.t_base_vendas (
    ID_MARCA INT64,
    MARCA STRING,
    ID_LINHA INT64,
    LINHA STRING,
    DATA_VENDA DATE,
    QTD_VENDA INT64
);

"""

# Executar a instrução SQL CREATE OR REPLACE
query_job_create_replace_raw = client.query(sql_create_replace_raw)
query_job_create_replace_raw.result()

# Substitua com as informações do seu projeto, dataset e tabela no BigQuery
dataset_id_raw = 'raw_data'
table_id_raw = 't_base_vendas'

# Construir a instrução SQL
sql_query_base_vendas = f"""
SELECT
  ID_MARCA,
  MARCA,
  ID_LINHA,
  LINHA,
  DATE(DATA_VENDA) AS DATA_VENDA,
  QTD_VENDA
FROM  transient.stg_base_vendas;
"""

# Executar a instrução SQL e salvar os resultados em um DataFrame
query_job_base_vendas = client.query(sql_query_base_vendas)
df_result_base_vendas = query_job_base_vendas.to_dataframe()

# Verificar se há dados para inserir
if not df_result_base_vendas.empty:
    # Imprimir o DataFrame com os resultados
    print("Carregando df:")
    # Carregar dados no BigQuery
    load_to_bigquery(df_result_base_vendas, dataset_id_raw, table_id_raw, credentials)
else:
    print(f'Nenhum novo dado para inserir na {dataset_id_raw}.{table_id_raw}')

###############################################################################################
#                               CARGA DO ARQUIVO NA TRUSTED                                  #
###############################################################################################

# Construir a instrução SQL CREATE OR REPLACE --> TRUNCATE STG  -- VERSÃO GRATIS NÃO DEIXA FAZER TRUNCATE 
sql_create_replace_trusted = f"""

CREATE OR REPLACE TABLE trusted.t_base_vendas_ano_mes (
  ANO_MES STRING,
  QTD_VENDA INT64
);

CREATE OR REPLACE TABLE trusted.t_base_vendas_marca_linha (
  MARCA STRING,
  LINHA STRING,
  QTD_VENDA INT64
);


CREATE OR REPLACE TABLE trusted.t_base_vendas_marca_ano_mes (
  ANO_MES STRING,
  MARCA STRING,
  QTD_VENDA INT64
);


CREATE OR REPLACE TABLE trusted.t_base_vendas_linha_ano_mes (
  ANO_MES STRING,
  LINHA STRING,
  QTD_VENDA INT64
);

"""

# Executar a instrução SQL CREATE OR REPLACE
query_job_create_replace_trusted = client.query(sql_create_replace_trusted)
query_job_create_replace_trusted.result()

# --a. Tabela 1: Consolidado de vendas por ano e mês;
dataset_id_trusted_ano_mes = 'trusted'
table_id_trusted_ano_mes = 't_base_vendas_ano_mes'

# Construir a instrução SQL
sql_query_mes_ano = f"""
SELECT
    FORMAT_DATE('%Y-%m', DATA_VENDA) AS ANO_MES
  , SUM(QTD_VENDA) as QTD_VENDA
FROM   raw_data.t_base_vendas
GROUP BY  1
ORDER BY 1  ;

"""

# Executar a instrução SQL e salvar os resultados em um DataFrame
query_job_trusted_ano_mes = client.query(sql_query_mes_ano)
df_result_trusted_ano_mes = query_job_trusted_ano_mes.to_dataframe()

# Verificar se há dados para inserir
if not df_result_trusted_ano_mes.empty:
    # Imprimir o DataFrame com os resultados
    print("Tabela 1: Consolidado de vendas por ano e mês:")
 
    # Carregar dados no BigQuery
    load_to_bigquery(df_result_trusted_ano_mes, dataset_id_trusted_ano_mes, table_id_trusted_ano_mes, credentials)

else:
    print(f"Nenhum novo dado para inserir na {dataset_id_trusted_ano_mes}.{table_id_trusted_ano_mes}.")

#--b. Tabela 2: Consolidado de vendas por marca e linha;
dataset_id_trusted_marca_linha = 'trusted'
table_id_trusted_marca_linha = 't_base_vendas_marca_linha'

# Construir a instrução SQL
sql_query_marca_linha = f"""
SELECT
    MARCA
  , LINHA
  , SUM(QTD_VENDA) as QTD_VENDA
FROM
  raw_data.t_base_vendas
  GROUP BY  1,2
  ORDER BY 1;

"""

# Executar a instrução SQL e salvar os resultados em um DataFrame
query_job_trusted_marca_linha = client.query(sql_query_marca_linha)
df_result_trusted_marca_linha = query_job_trusted_marca_linha.to_dataframe()

# Verificar se há dados para inserir
if not df_result_trusted_marca_linha.empty:
    # Imprimir o DataFrame com os resultados
    print("Tabela 2: Consolidado de vendas por marca e linha:")
 
    # Carregar dados no BigQuery
    load_to_bigquery(df_result_trusted_marca_linha, dataset_id_trusted_marca_linha, table_id_trusted_marca_linha, credentials)

else:
    print(F"Nenhum novo dado para inserir na {dataset_id_trusted_marca_linha}.{table_id_trusted_marca_linha}.")

# --c. Tabela 3: Consolidado de vendas por marca, ano e mês;
dataset_id_trusted_marca_ano_mes = 'trusted'
table_id_trusted_marca_ano_mes = 't_base_vendas_marca_ano_mes'

# Construir a instrução SQL
sql_query_marca_ano_mes = f"""
SELECT
    FORMAT_DATE('%Y-%m', DATA_VENDA) AS ANO_MES
  , MARCA
  , SUM(QTD_VENDA) as QTD_VENDA
FROM   raw_data.t_base_vendas
GROUP BY  1, 2
ORDER BY 1, 2;

"""

# Executar a instrução SQL e salvar os resultados em um DataFrame
query_job_trusted_marca_ano_mes = client.query(sql_query_marca_ano_mes)
df_result_trusted_marca_ano_mes = query_job_trusted_marca_ano_mes.to_dataframe()

# Verificar se há dados para inserir
if not df_result_trusted_marca_ano_mes.empty:
    # Imprimir o DataFrame com os resultados
    print("Tabela 3: Consolidado de vendas por marca, ano e mês:")
 
    # Carregar dados no BigQuery
    load_to_bigquery(df_result_trusted_marca_ano_mes, dataset_id_trusted_marca_ano_mes, table_id_trusted_marca_ano_mes, credentials)

else:
    print(f"Nenhum novo dado para inserir na {dataset_id_trusted_marca_ano_mes}.{table_id_trusted_marca_ano_mes}.")

# --d. Tabela 4: Consolidado de vendas por linha, ano e mês;
dataset_id_trusted_linha_ano_mes = 'trusted'
table_id_trusted_linha_ano_mes = 't_base_vendas_linha_ano_mes'

# Construir a instrução SQL
sql_query_linha_ano_mes = f"""
SELECT
    FORMAT_DATE('%Y-%m', DATA_VENDA) AS ANO_MES
  , LINHA
  , SUM(QTD_VENDA) as QTD_VENDA
FROM   raw_data.t_base_vendas
GROUP BY  1, 2
ORDER BY 1, 2;

"""

# Executar a instrução SQL e salvar os resultados em um DataFrame
query_job_trusted_linha_ano_mes = client.query(sql_query_linha_ano_mes)
df_result_trusted_linha_ano_mes = query_job_trusted_linha_ano_mes.to_dataframe()

# Verificar se há dados para inserir
if not df_result_trusted_linha_ano_mes.empty:
    # Imprimir o DataFrame com os resultados
    print("Tabela 4: Consolidado de vendas por linha, ano e mês:")
 
    # Carregar dados no BigQuery
    load_to_bigquery(df_result_trusted_linha_ano_mes, dataset_id_trusted_linha_ano_mes, table_id_trusted_linha_ano_mes, credentials)
         